In [1]:
from functools import partial

import numpy as np


l1_norm = partial(np.linalg.norm, ord=1, axis=-1)


class FuzzyART(object):
    """
    Fuzzy ART
    
    An unsupervised clustering algorithm 
    """
    def __init__(self, alpha=1.0, gamma=0.01, rho=0.5, complement_coding=True):
        """        
        :param alpha: learning rate [0,1] 
        :param gamma: regularization term >0
        :param rho: vigilance [0,1]
        :param complement_coding: use complement coding scheme for inputs
        """
        self.alpha = alpha  
        self.beta = 1 - alpha
        self.gamma = gamma  
        self.rho = rho  
        self.complement_coding = complement_coding
        print("Learning rate =", alpha)
        print("Vigilance parameter =", rho)
        print("compliment coding = True")
        self.w = None

    def _init_weights(self, x):
        self.w = np.atleast_2d(x)
        print("weights:", self.w)
    def _complement_code(self, x):
        if self.complement_coding:
            return np.hstack((x, 1-x))
        else:
            return x

    def _add_category(self, x):
        self.w = np.vstack((self.w, x))

    def _match_category(self, x):
        fuzzy_weights = np.minimum(x, self.w)
        #print("fuzzy weights =", fuzzy_weights)
        fuzzy_norm = l1_norm(fuzzy_weights)
        scores = fuzzy_norm / (self.gamma + l1_norm(self.w))
        threshold = fuzzy_norm / l1_norm(x) >= self.rho
        if np.all(threshold == False):
            return -1
        else:
            return np.argmax(scores * threshold.astype(int))

    def train(self, x, epochs=10):
        """        
        :param x: 2d array of size (samples, features), where all features are
         in [0, 1]
        :param epochs: number of training epochs, the training samples are 
        shuffled after each epoch  
        :return: self
        """
        samples = self._complement_code(np.atleast_2d(x))

        if self.w is None:
            self._init_weights(samples[0])

        for epoch in range(epochs):
            print("epoch=", epoch)
            for sample in np.random.permutation(samples):
                category = self._match_category(sample)
                if category == -1:
                    self._add_category(sample)
                else:
                    w = self.w[category]
                    self.w[category] = (self.alpha * np.minimum(sample, w) +
                                        self.beta * w)
        return self

    def test(self, x):
        """        
        :param x: 2d array of size (samples, features), where all features are
         in [0, 1] 
        :return: category IDs for each provided sample
        """
        samples = self._complement_code(np.atleast_2d(x))

        categories = np.zeros(len(samples))
        for i, sample in enumerate(samples):
            categories[i] = self._match_category(sample)
        return categories


class FuzzyARTMAP(object):
    """
    Fuzzy ARTMAP
    
    A supervised version of FuzzyART
    """

    def __init__(self, alpha=1.0, gamma=0.01, rho=0.5, epsilon=-0.0001,
                 complement_coding=True):
        """        
        :param alpha: learning rate [0,1] 
        :param gamma: regularization term >0
        :param rho: vigilance [0,1]
        :param epsilon: match tracking [-1,1]
        :param complement_coding: use complement coding scheme for inputs
        """
        self.alpha = alpha  # learning rate
        self.beta = 1 - alpha
        self.gamma = gamma  # choice parameter
        self.rho = rho  # vigilance
        self.epsilon = epsilon  # match tracking
        self.complement_coding = complement_coding

        self.w = None
        self.out_w = None
        self.n_classes = 0

    def _init_weights(self, x, y):
        self.w = np.atleast_2d(x)
        self.out_w = np.zeros((1, self.n_classes))
        self.out_w[0, y] = 1

    def _complement_code(self, x):
        if self.complement_coding:
            return np.hstack((x, 1-x))
        else:
            return x

    def _add_category(self, x, y):
        self.w = np.vstack((self.w, x))
        self.out_w = np.vstack((self.out_w, np.zeros(self.n_classes)))
        self.out_w[-1, y] = 1

    def _match_category(self, x, y=None):
        _rho = self.rho
        fuzzy_weights = np.minimum(x, self.w)
        fuzzy_norm = l1_norm(fuzzy_weights)
        scores = fuzzy_norm + (1 - self.gamma) * (l1_norm(x) + l1_norm(self.w))
        norms = fuzzy_norm / l1_norm(x)

        threshold = norms >= _rho
        while not np.all(threshold == False):
            y_ = np.argmax(scores * threshold.astype(int))

            if y is None or self.out_w[y_, y] == 1:
                return y_
            else:
                _rho = norms[y_] + self.epsilon
                norms[y_] = 0
                threshold = norms >= _rho
        return -1

    def train(self, x, y, epochs=10):
        """        
        :param x: 2d array of size (samples, features), where all features are
         in [0, 1]
        :param y: 1d array of size (samples,) containing the class label of each
        sample
        :param epochs: number of training epochs, the training samples are 
        shuffled after each epoch  
        :return: self
        """
        samples = self._complement_code(np.atleast_2d(x))
        self.n_classes = len(set(y))

        if self.w is None:
            self._init_weights(samples[0], y[0])

        idx = np.arange(len(samples), dtype=np.uint32)

        for epoch in range(epochs):
            
            idx = np.random.permutation(idx)
            for sample, label in zip(samples[idx], y[idx]):
                category = self._match_category(sample, label)
                if category == -1:
                    self._add_category(sample, label)
                else:
                    w = self.w[category]
                    self.w[category] = (self.alpha * np.minimum(sample, w) +
                                        self.beta * w)
        return self

    def test(self, x):
        """        
        :param x: 2d array of size (samples, features), where all features are
         in [0, 1] 
        :return: class label for each provided sample
        """
        samples = self._complement_code(np.atleast_2d(x))

        labels = np.zeros(len(samples))
        for i, sample in enumerate(samples):
            category = self._match_category(sample)
            labels[i] = np.argmax(self.out_w[category])
        return labels



In [2]:
import numpy as np # linear algebra
import pandas as pd
data=pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")
data=data.drop(['Unnamed: 32', 'id'], axis = 1)
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
data['diagnosis']=label.fit_transform(data["diagnosis"])
y=data.diagnosis.values
#x_data=data.drop(['id'], axis=0)
x_data=data.drop(['diagnosis'], axis=1)

In [3]:
import numpy as np
if __name__ == '__main__':
    import sklearn.datasets as ds
    wisc = ds.load_breast_cancer()
    data=[x_data]
    data = wisc['data'] / np.max(wisc['data'], axis=0)
    net = FuzzyART(alpha=0.5, rho=0.75)
    net.train(data, epochs=20)
    target=wisc['target']
    print(net.w.shape)
    print(net.test(data).astype(int))
    print("targert=", wisc['target'])

Learning rate = 0.5
Vigilance parameter = 0.75
compliment coding = True
weights: [[0.63998577 0.26425662 0.65145889 0.4002399  0.7246022  0.80370585
  0.70313964 0.73111332 0.79572368 0.80777915 0.3811347  0.18532242
  0.39076433 0.28292143 0.20555734 0.36218612 0.13568182 0.30062512
  0.38036732 0.20754021 0.70421754 0.34981833 0.73487261 0.47461213
  0.72866128 0.62911153 0.56861022 0.91202749 0.69313046 0.57301205
  0.36001423 0.73574338 0.34854111 0.5997601  0.2753978  0.19629415
  0.29686036 0.26888668 0.20427632 0.19222085 0.6188653  0.81467758
  0.60923567 0.71707857 0.79444266 0.63781388 0.86431818 0.69937488
  0.61963268 0.79245979 0.29578246 0.65018167 0.26512739 0.52538787
  0.27133872 0.37088847 0.43138978 0.08797251 0.30686954 0.42698795]]
epoch= 0
epoch= 1
epoch= 2
epoch= 3
epoch= 4
epoch= 5
epoch= 6
epoch= 7
epoch= 8
epoch= 9
epoch= 10
epoch= 11
epoch= 12
epoch= 13
epoch= 14
epoch= 15
epoch= 16
epoch= 17
epoch= 18
epoch= 19
(86, 60)
[ 0  2 52 24 85 11 46 61 11 16 27 37  

In [4]:
print(np.shape(target))

(569,)


In [5]:
y=y.T 
target2=np.logical_not(y)
print(target2)
target1=np.logical_not(target)
target1=np.logical_not(target1)
print(target1)

[False False False False False False False False False False False False
 False False False False False False False  True  True  True False False
 False False False False False False False False False False False False
 False  True False False False False False False False False  True False
  True  True  True  True  True False False  True False False  True  True
  True  True False  True False False  True  True  True  True False  True
 False False  True False  True False False  True  True  True False False
  True False False False  True  True  True False  True  True False False
  True  True  True False False  True  True  True  True False  True  True
 False  True  True  True  True  True  True  True  True False False False
  True False False  True  True  True False False  True False  True False
 False  True False False  True  True False  True  True False  True  True
  True  True False  True  True  True  True  True  True  True  True  True
 False  True  True  True  True False False  True Fa

In [6]:
#accuracy: 
result=np.array_equal(target2, target1)
if result==True: 
    acc="100%"
else: acc="not 100"
print("Accuracy of ART is", acc)

Accuracy of ART is 100%
